# **Downloading Dataset**

In [ ]:
dataset_folder_name = "Dataset"

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  40907      0 --:--:-- --:--:-- --:--:-- 40907
OK
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 14.0 MB of archives.
After this operation, 31.2 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../gcsfuse_1.0.0_amd64.deb ...
Unpacking gcsfuse (1.0.0) ...
Setting up gcsfuse (1.0.0) ...


In [ ]:
!mkdir $dataset_folder_name
!gcsfuse --implicit-dirs i-care-2.0.physionet.org $dataset_folder_name

I0629 18:59:38.406204 2023/06/29 18:59:38.406170 Start gcsfuse/1.0.0 (Go version go1.20.4) for app "" using mount point: /content/Dataset


# **helper_code.py**

In [ ]:
#!/usr/bin/env python

# Do *not* edit this script.
# These are helper functions that you can use with your code.
# Check the example code to see how to import these functions to your code.

import os, numpy as np, scipy as sp, scipy.io

### Challenge data I/O functions

# Find the folders with data files.
def find_data_folders(root_folder):
    data_folders = list()
    for x in sorted(os.listdir(root_folder)):
        data_folder = os.path.join(root_folder, x)
        if os.path.isdir(data_folder):
            data_file = os.path.join(data_folder, x + '.txt')
            if os.path.isfile(data_file):
                data_folders.append(x)
    return sorted(data_folders)

# Load the patient metadata: age, sex, etc.
def load_challenge_data(data_folder, patient_id):
    patient_metadata_file = os.path.join(data_folder, patient_id, patient_id + '.txt')
    patient_metadata = load_text_file(patient_metadata_file)
    return patient_metadata

# Find the record names.
def find_recording_files(data_folder, patient_id):
    record_names = list()
    patient_folder = os.path.join(data_folder, patient_id)
    for file_name in sorted(os.listdir(patient_folder)):
        if not file_name.startswith('.') and file_name.endswith('.hea'):
            root, ext = os.path.splitext(file_name)
            record_name = '_'.join(root.split('_')[:-1])
            record_names.append(record_name)
    return sorted(record_names)

# Load the WFDB data for the Challenge (but not all possible WFDB files).
def load_recording_data(record_name, check_values=True):
    # Allow either the record name or the header filename.
    root, ext = os.path.splitext(record_name)
    if ext=='':
        header_file = record_name + '.hea'
    else:
        header_file = record_name

    # Load the header file.
    if not os.path.isfile(header_file):
        raise FileNotFoundError('{} recording not found.'.format(record_name))

    with open(header_file, 'r') as f:
        header = [l.strip() for l in f.readlines() if l.strip()]

    # Parse the header file.
    record_name = None
    num_signals = None
    sampling_frequency = None
    num_samples = None
    signal_files = list()
    gains = list()
    offsets = list()
    channels = list()
    initial_values = list()
    checksums = list()

    for i, l in enumerate(header):
        arrs = [arr.strip() for arr in l.split(' ')]
        # Parse the record line.
        if i==0:
            record_name = arrs[0]
            num_signals = int(arrs[1])
            sampling_frequency = float(arrs[2])
            num_samples = int(arrs[3])
        # Parse the signal specification lines.
        elif not l.startswith('#') or len(l.strip()) == 0:
            signal_file = arrs[0]
            gain = float(arrs[2].split('/')[0])
            offset = int(arrs[4])
            initial_value = int(arrs[5])
            checksum = int(arrs[6])
            channel = arrs[8]
            signal_files.append(signal_file)
            gains.append(gain)
            offsets.append(offset)
            initial_values.append(initial_value)
            checksums.append(checksum)
            channels.append(channel)

    # Check that the header file only references one signal file. WFDB format allows for multiple signal files, but, for
    # simplicity, we have not done that here.
    num_signal_files = len(set(signal_files))
    if num_signal_files!=1:
        raise NotImplementedError('The header file {}'.format(header_file) \
            + ' references {} signal files; one signal file expected.'.format(num_signal_files))

    # Load the signal file.
    head, tail = os.path.split(header_file)
    signal_file = os.path.join(head, list(signal_files)[0])
    data = np.asarray(sp.io.loadmat(signal_file)['val'])

    # Check that the dimensions of the signal data in the signal file is consistent with the dimensions for the signal data given
    # in the header file.
    num_channels = len(channels)
    if np.shape(data)!=(num_channels, num_samples):
        raise ValueError('The header file {}'.format(header_file) \
            + ' is inconsistent with the dimensions of the signal file.')

    # Check that the initial value and checksums in the signal file are consistent with the initial value and checksums in the
    # header file.
    if check_values:
        for i in range(num_channels):
            if data[i, 0]!=initial_values[i]:
                raise ValueError('The initial value in header file {}'.format(header_file) \
                    + ' is inconsistent with the initial value for channel {} in the signal data'.format(channels[i]))
            if np.sum(data[i, :])!=checksums[i]:
                raise ValueError('The checksum in header file {}'.format(header_file) \
                    + ' is inconsistent with the checksum value for channel {} in the signal data'.format(channels[i]))

    # Rescale the signal data using the gains and offsets.
    rescaled_data = np.zeros(np.shape(data), dtype=np.float32)
    for i in range(num_channels):
        rescaled_data[i, :] = (data[i, :]-offsets[i])/gains[i]

    return rescaled_data, channels, sampling_frequency

# Choose the channels.
def reduce_channels(current_data, current_channels, requested_channels):
    if current_channels == requested_channels:
        reduced_data = current_data
        reduced_channels = current_channels
    else:
        reduced_indices = [current_channels.index(channel) for channel in requested_channels if channel in current_channels]
        reduced_channels = [current_channels[i] for i in reduced_indices]
        reduced_data = current_data[reduced_indices, :]
    return reduced_data, reduced_channels

# Choose the channels.
def expand_channels(current_data, current_channels, requested_channels):
    if current_channels == requested_channels:
        expanded_data = current_data
    else:
        num_current_channels, num_samples = np.shape(current_data)
        num_requested_channels = len(requested_channels)
        expanded_data = np.zeros((num_requested_channels, num_samples))
        for i, channel in enumerate(requested_channels):
            if channel in current_channels:
                j = current_channels.index(channel)
                expanded_data[i, :] = current_data[j, :]
            else:
                expanded_data[i, :] = float('nan')
    return expanded_data

### Helper Challenge data I/O functions

# Load text file as a string.
def load_text_file(filename):
    with open(filename, 'r') as f:
        data = f.read()
    return data

# Get a variable from the patient metadata.
def get_variable(text, variable_name, variable_type):
    variable = None
    for l in text.split('\n'):
        if l.startswith(variable_name):
            variable = ':'.join(l.split(':')[1:]).strip()
            variable = cast_variable(variable, variable_type)
            return variable

# Get the patient ID variable from the patient data.
def get_patient_id(string):
    return get_variable(string, 'Patient', str)

# Get the patient ID variable from the patient data.
def get_hospital(string):
    return get_variable(string, 'Hospital', str)

# Get the age variable (in years) from the patient data.
def get_age(string):
    return get_variable(string, 'Age', int)

# Get the sex variable from the patient data.
def get_sex(string):
    return get_variable(string, 'Sex', str)

# Get the ROSC variable (in minutes) from the patient data.
def get_rosc(string):
    return get_variable(string, 'ROSC', int)

# Get the OHCA variable from the patient data.
def get_ohca(string):
    return get_variable(string, 'OHCA', bool)

# Get the shockable rhythm variable from the patient data.
def get_shockable_rhythm(string):
    return get_variable(string, 'Shockable Rhythm', bool)

# Get the TTM variable (in Celsius) from the patient data.
def get_ttm(string):
    return get_variable(string, 'TTM', int)

# Get the Outcome variable from the patient data.
def get_outcome(string):
    variable = get_variable(string, 'Outcome', str)
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    if variable == 'Good':
        variable = 0
    elif variable == 'Poor':
        variable = 1
    return variable

# Get the Outcome probability variable from the patient data.
def get_outcome_probability(string):
    variable = sanitize_scalar_value(get_variable(string, 'Outcome Probability', str))
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the CPC variable from the patient data.
def get_cpc(string):
    variable = sanitize_scalar_value(get_variable(string, 'CPC', str))
    if variable is None or is_nan(variable):
        raise ValueError('No CPC score available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the utility frequency (in Hertz) from the recording data.
def get_utility_frequency(string):
    return get_variable(string, '#Utility frequency', int)

# Get the start time (in hh:mm:ss format) from the recording data.
def get_start_time(string):
    variable = get_variable(string, '#Start time', str)
    times = tuple(int(value) for value in variable.split(':'))
    return times

# Get the end time (in hh:mm:ss format) from the recording data.
def get_end_time(string):
    variable = get_variable(string, '#End time', str)
    times = tuple(int(value) for value in variable.split(':'))
    return times

# Convert seconds to days, hours, minutes, seconds.
def convert_seconds_to_hours_minutes_seconds(seconds):
    hours = int(seconds/3600 - 24*days)
    minutes = int(seconds/60 - 24*60*days - 60*hours)
    seconds = int(seconds - 24*3600*days - 3600*hours - 60*minutes)
    return hours, minutes, seconds

# Convert hours, minutes, and seconds to seconds.
def convert_hours_minutes_seconds_to_seconds(hours, minutes, seconds):
    return 3600*hours + 60*minutes + seconds

### Challenge label and output I/O functions


# Save the Challenge outputs for one file.
def save_challenge_outputs(filename, patient_id, outcome, outcome_probability, cpc):
    # Sanitize values, e.g., in case they are a singleton array.
    outcome = sanitize_boolean_value(outcome)
    outcome_probability = sanitize_scalar_value(outcome_probability)
    cpc = sanitize_scalar_value(cpc)

    # Format Challenge outputs.
    patient_string = 'Patient: {}'.format(patient_id)
    if outcome == 0:
        outcome = 'Good'
    elif outcome == 1:
        outcome = 'Poor'
    outcome_string = 'Outcome: {}'.format(outcome)
    outcome_probability_string = 'Outcome Probability: {:.3f}'.format(outcome_probability)
    cpc_string = 'CPC: {:.3f}'.format(cast_int_if_int_else_float(cpc))
    output_string = patient_string + '\n' + \
        outcome_string + '\n' + outcome_probability_string + '\n' + cpc_string + '\n'

    # Write the Challenge outputs.
    if filename is not None:
        with open(filename, 'w') as f:
            f.write(output_string)

    return output_string

### Other helper functions

# Check if a variable is a number or represents a number.
def is_number(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False

# Check if a variable is an integer or represents an integer.
def is_integer(x):
    if is_number(x):
        return float(x).is_integer()
    else:
        return False

# Check if a variable is a boolean or represents a boolean.
def is_boolean(x):
    if (is_number(x) and float(x)==0) or (remove_extra_characters(x) in ('False', 'false', 'FALSE', 'F', 'f')):
        return True
    elif (is_number(x) and float(x)==1) or (remove_extra_characters(x) in ('True', 'true', 'TRUE', 'T', 't')):
        return True
    else:
        return False

# Check if a variable is a finite number or represents a finite number.
def is_finite_number(x):
    if is_number(x):
        return np.isfinite(float(x))
    else:
        return False

# Check if a variable is a NaN (not a number) or represents a NaN.
def is_nan(x):
    if is_number(x):
        return np.isnan(float(x))
    else:
        return False

# Remove any quotes, brackets (for singleton arrays), and/or invisible characters.
def remove_extra_characters(x):
    return str(x).replace('"', '').replace("'", "").replace('[', '').replace(']', '').replace(' ', '').strip()

# Sanitize boolean values.
def sanitize_boolean_value(x):
    x = remove_extra_characters(x)
    if (is_number(x) and float(x)==0) or (remove_extra_characters(x) in ('False', 'false', 'FALSE', 'F', 'f')):
        return 0
    elif (is_number(x) and float(x)==1) or (remove_extra_characters(x) in ('True', 'true', 'TRUE', 'T', 't')):
        return 1
    else:
        return float('nan')

# Sanitize integer values.
def sanitize_integer_value(x):
    x = remove_extra_characters(x)
    if is_integer(x):
        return int(float(x))
    else:
        return float('nan')

# Sanitize scalar values.
def sanitize_scalar_value(x):
    x = remove_extra_characters(x)
    if is_number(x):
        return float(x)
    else:
        return float('nan')

# Cast a value to a particular type.
def cast_variable(variable, variable_type, preserve_nan=True):
    if preserve_nan and is_nan(variable):
        variable = float('nan')
    else:
        if variable_type == bool:
            variable = sanitize_boolean_value(variable)
        elif variable_type == int:
            variable = sanitize_integer_value(variable)
        elif variable_type == float:
            variable = sanitize_scalar_value(variable)
        else:
            variable = variable_type(variable)
    return variable

# Cast a value to an integer if the value is an integer, a float if the value is a non-integer float, and itself otherwise.
def cast_int_if_int_else_float(x):
    if is_integer(x):
        return int(float(x))
    elif is_number(x):
        return float(x)
    else:
        return x


# **Teamcode Helper for train**

In [ ]:
import joblib
def save_challenge_model(model_folder, imputer, outcome_model, cpc_model):
    d = {'imputer': imputer, 'outcome_model': outcome_model, 'cpc_model': cpc_model}
    filename = os.path.join(model_folder, 'models.sav')
    joblib.dump(d, filename, protocol=0)

In [ ]:
!pip install -q mne
import mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.3 MB/s eta 0:00:00


In [ ]:
# Preprocess data.
def preprocess_data(data, sampling_frequency, utility_frequency):
    # Define the bandpass frequencies.
    passband = [0.1, 30.0]

    # Promote the data to double precision because these libraries expect double precision.
    data = np.asarray(data, dtype=np.float64)

    # If the utility frequency is between bandpass frequencies, then apply a notch filter.
    if utility_frequency is not None and passband[0] <= utility_frequency <= passband[1]:
        data = mne.filter.notch_filter(data, sampling_frequency, utility_frequency, n_jobs=4, verbose='error')

    # Apply a bandpass filter.
    data = mne.filter.filter_data(data, sampling_frequency, passband[0], passband[1], n_jobs=4, verbose='error')

    # Resample the data.
    if sampling_frequency % 2 == 0:
        resampling_frequency = 128
    else:
        resampling_frequency = 125
    lcm = np.lcm(int(round(sampling_frequency)), int(round(resampling_frequency)))
    up = int(round(lcm / sampling_frequency))
    down = int(round(lcm / resampling_frequency))
    resampling_frequency = sampling_frequency * up / down
    data = scipy.signal.resample_poly(data, up, down, axis=1)

    # Scale the data to the interval [-1, 1].
    min_value = np.min(data)
    max_value = np.max(data)
    if min_value != max_value:
        data = 2.0 / (max_value - min_value) * (data - 0.5 * (min_value + max_value))
    else:
        data = 0 * data

    return data, resampling_frequency

# Extract features.
def get_features(data_folder, patient_id):
    # Load patient data.
    patient_metadata = load_challenge_data(data_folder, patient_id)
    recording_ids = find_recording_files(data_folder, patient_id)
    num_recordings = len(recording_ids)

    # Extract patient features.
    patient_features = get_patient_features(patient_metadata)

    # Extract EEG features.
    eeg_channels = ['F3', 'P3', 'F4', 'P4']
    group = 'EEG'

    if num_recordings > 0:
        recording_id = recording_ids[-1]
        recording_location = os.path.join(data_folder, patient_id, '{}_{}'.format(recording_id, group))
        if os.path.exists(recording_location + '.hea'):
            data, channels, sampling_frequency = load_recording_data(recording_location)
            utility_frequency = get_utility_frequency(recording_location + '.hea')

            if all(channel in channels for channel in eeg_channels):
                data, channels = reduce_channels(data, channels, eeg_channels)
                data, sampling_frequency = preprocess_data(data, sampling_frequency, utility_frequency)
                data = np.array([data[0, :] - data[1, :], data[2, :] - data[3, :]]) # Convert to bipolar montage: F3-P3 and F4-P4
                eeg_features = get_eeg_features(data, sampling_frequency).flatten()
            else:
                eeg_features = float('nan') * np.ones(8) # 2 bipolar channels * 4 features / channel
        else:
            eeg_features = float('nan') * np.ones(8) # 2 bipolar channels * 4 features / channel
    else:
        eeg_features = float('nan') * np.ones(8) # 2 bipolar channels * 4 features / channel

    # Extract ECG features.
    ecg_channels = ['ECG', 'ECGL', 'ECGR', 'ECG1', 'ECG2']
    group = 'ECG'

    if num_recordings > 0:
        recording_id = recording_ids[0]
        recording_location = os.path.join(data_folder, patient_id, '{}_{}'.format(recording_id, group))
        if os.path.exists(recording_location + '.hea'):
            data, channels, sampling_frequency = load_recording_data(recording_location)
            utility_frequency = get_utility_frequency(recording_location + '.hea')

            data, channels = reduce_channels(data, channels, ecg_channels)
            data, sampling_frequency = preprocess_data(data, sampling_frequency, utility_frequency)
            features = get_ecg_features(data)
            ecg_features = expand_channels(features, channels, ecg_channels).flatten()
        else:
            ecg_features = float('nan') * np.ones(10) # 5 channels * 2 features / channel
    else:
        ecg_features = float('nan') * np.ones(10) # 5 channels * 2 features / channel

    # Extract features.
    return np.hstack((patient_features, eeg_features, ecg_features))

# Extract patient features from the data.
def get_patient_features(data):
    age = get_age(data)
    sex = get_sex(data)
    rosc = get_rosc(data)
    ohca = get_ohca(data)
    shockable_rhythm = get_shockable_rhythm(data)
    ttm = get_ttm(data)

    sex_features = np.zeros(2, dtype=int)
    if sex == 'Female':
        female = 1
        male   = 0
        other  = 0
    elif sex == 'Male':
        female = 0
        male   = 1
        other  = 0
    else:
        female = 0
        male   = 0
        other  = 1

    features = np.array((age, female, male, other, rosc, ohca, shockable_rhythm, ttm))

    return features

# Extract features from the EEG data.
def get_eeg_features(data, sampling_frequency):
    num_channels, num_samples = np.shape(data)

    if num_samples > 0:
        delta_psd, _ = mne.time_frequency.psd_array_welch(data, sfreq=sampling_frequency,  fmin=0.5,  fmax=8.0, verbose=False)
        theta_psd, _ = mne.time_frequency.psd_array_welch(data, sfreq=sampling_frequency,  fmin=4.0,  fmax=8.0, verbose=False)
        alpha_psd, _ = mne.time_frequency.psd_array_welch(data, sfreq=sampling_frequency,  fmin=8.0, fmax=12.0, verbose=False)
        beta_psd,  _ = mne.time_frequency.psd_array_welch(data, sfreq=sampling_frequency, fmin=12.0, fmax=30.0, verbose=False)

        delta_psd_mean = np.nanmean(delta_psd, axis=1)
        theta_psd_mean = np.nanmean(theta_psd, axis=1)
        alpha_psd_mean = np.nanmean(alpha_psd, axis=1)
        beta_psd_mean  = np.nanmean(beta_psd,  axis=1)
    else:
        delta_psd_mean = theta_psd_mean = alpha_psd_mean = beta_psd_mean = float('nan') * np.ones(num_channels)

    features = np.array((delta_psd_mean, theta_psd_mean, alpha_psd_mean, beta_psd_mean)).T

    return features

# Extract features from the ECG data.
def get_ecg_features(data):
    num_channels, num_samples = np.shape(data)

    if num_samples > 0:
        mean = np.mean(data, axis=1)
        std  = np.std(data, axis=1)
    elif num_samples == 1:
        mean = np.mean(data, axis=1)
        std  = float('nan') * np.ones(num_channels)
    else:
        mean = float('nan') * np.ones(num_channels)
        std = float('nan') * np.ones(num_channels)

    features = np.array((mean, std)).T

    return features

# **TeamCode Helper for Inference**

In [ ]:
def load_challenge_models(model_folder, verbose):
    filename = os.path.join(model_folder, 'models.sav')
    return joblib.load(filename)

# Run your trained models. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_models(models, data_folder, patient_id, verbose):
    imputer = models['imputer']
    outcome_model = models['outcome_model']
    cpc_model = models['cpc_model']

    # Extract features.
    features = get_features(data_folder, patient_id)
    features = features.reshape(1, -1)

    # Impute missing data.
    features = imputer.transform(features)

    # Apply models to features.
    outcome = outcome_model.predict(features)[0]
    outcome_probability = outcome_model.predict_proba(features)[0, 1]
    cpc = cpc_model.predict(features)[0]

    # Ensure that the CPC score is between (or equal to) 1 and 5.
    cpc = np.clip(cpc, 1, 5)

    return outcome, outcome_probability, cpc

# **Generating Train/val/test folders**

In [ ]:
data_folder = f"/content/{dataset_folder_name}/training" # input
patientId_label_dic = {}

patient_id_list = find_data_folders(data_folder)
for patient_id in patient_id_list:
  patient_folder = os.path.join(data_folder, patient_id)

  ## find label
  patient_metadata_file = os.path.join(data_folder, patient_id, patient_id + '.txt')
  patient_metadata = load_text_file(patient_metadata_file)
  label = get_outcome(patient_metadata)

  patientId_label_dic[patient_folder] = label

In [ ]:
import pandas as pd

# convert dictionary to pandas dataframe
patient_label_df = pd.DataFrame(list(patientId_label_dic.items()), columns=['patient_data_path', 'label'])

In [ ]:
seed = 5
patient_label_df = patient_label_df.sample(frac=1, random_state=seed)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the original dataset into train and test sets
train_val_df, test_df = train_test_split(patient_label_df, test_size=0.2, stratify=patient_label_df['label'], random_state=seed)

# Split the training set into train and validation sets
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['label'], random_state=seed)

In [ ]:
# patient_label_df['label'].value_counts()
# train_df['label'].value_counts()
# val_df['label'].value_counts()
# test_df['label'].value_counts()

In [ ]:
data_path = "/content/DATA"
train_val_path = f"{data_path}/train_val"
train_path = f"{data_path}/train"
val_path = f"{data_path}/val"
test_path = f"{data_path}/test"

!mkdir -p $data_path
!mkdir -p $train_val_path
!mkdir -p $train_path
!mkdir -p $val_path
!mkdir -p $test_path

In [ ]:
def count_folders(path):
    folder_count = 0

    for _, dirs, _ in os.walk(path):
        folder_count += len(dirs)

    return folder_count

In [ ]:
# for index, row in train_val_df.iterrows():
#     patient_data_path = row['patient_data_path']
#     !ln -s $patient_data_path $train_val_path



# assert count_folders(train_val_path) == len(train_val_df)

for index, row in train_df.iterrows():
    patient_data_path = row['patient_data_path']
    !ln -s $patient_data_path $train_path

assert count_folders(train_path) == len(train_df)


# for index, row in val_df.iterrows():
#     patient_data_path = row['patient_data_path']
#     !ln -s $patient_data_path $val_path

# assert count_folders(val_path) == len(val_df)

In [ ]:
for index, row in test_df.iterrows():
    patient_data_path = row['patient_data_path']
    !ln -s $patient_data_path $test_path

assert count_folders(test_path) == len(test_df)

# **Train Model**

In [ ]:
# python train_model.py training_data model

In [ ]:
data_folder = train_path
model_folder = "/content/model"

In [ ]:
patient_ids = find_data_folders(data_folder)
num_patients = len(patient_ids)

if num_patients==0:
    raise FileNotFoundError('No data was provided.')

# Create a folder for the model if it does not already exist.
os.makedirs(model_folder, exist_ok=True)

# Extract the features and labels.
print('Extracting features and labels from the Challenge data...')

features = list()
outcomes = list()
cpcs = list()

for i in range(num_patients):
    print('    {}/{}...'.format(i+1, num_patients))

    current_features = get_features(data_folder, patient_ids[i])
    features.append(current_features)

    # Extract labels.
    patient_metadata = load_challenge_data(data_folder, patient_ids[i])
    current_outcome = get_outcome(patient_metadata)
    outcomes.append(current_outcome)
    current_cpc = get_cpc(patient_metadata)
    cpcs.append(current_cpc)

features = np.vstack(features)
outcomes = np.vstack(outcomes)
cpcs = np.vstack(cpcs)

Extracting features and labels from the Challenge data...
    1/388...
    2/388...
    3/388...
    4/388...
    5/388...
    6/388...
    7/388...
    8/388...
    9/388...
    10/388...
    11/388...
    12/388...
    13/388...
    14/388...
    15/388...
    16/388...
    17/388...
    18/388...
    19/388...
    20/388...
    21/388...
    22/388...
    23/388...
    24/388...
    25/388...
    26/388...
    27/388...
    28/388...
    29/388...
    30/388...
    31/388...
    32/388...
    33/388...
    34/388...
    35/388...
    36/388...
    37/388...
    38/388...
    39/388...
    40/388...
    41/388...
    42/388...
    43/388...
    44/388...
    45/388...
    46/388...
    47/388...
    48/388...
    49/388...
    50/388...
    51/388...
    52/388...
    53/388...
    54/388...
    55/388...
    56/388...
    57/388...
    58/388...
    59/388...
    60/388...
    61/388...
    62/388...
    63/388...
    64/388...
    65/388...
    66/388...
    67/388...
    68/388...

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Train the models.
print('Training the Challenge model on the Challenge data...')

# Define parameters for random forest classifier and regressor.
n_estimators   = 123  # Number of trees in the forest.
max_leaf_nodes = 456  # Maximum number of leaf nodes in each tree.
random_state   = 789  # Random state; set for reproducibility.

# Impute any missing features; use the mean value by default.
imputer = SimpleImputer().fit(features)

# Train the models.
features = imputer.transform(features)
outcome_model = RandomForestClassifier(
    n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes.ravel())
cpc_model = RandomForestRegressor(
    n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, cpcs.ravel())



Training the Challenge model on the Challenge data...


In [ ]:
# Save the models.
save_challenge_model(model_folder, imputer, outcome_model, cpc_model)

# **Inference on test data**

In [ ]:
# python run_model.py model test_data test_outputs

In [ ]:
data_folder = test_path
output_folder = "/content/output"

In [ ]:
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Load model(s).
print('Loading the Challenge models...')

# You can use this function to perform tasks, such as loading your models, that you only need to perform once.
models = load_challenge_models(model_folder, True) ### Teams: Implement this function!!!

# Find the Challenge data.
print('Finding the Challenge data...')

patient_ids = find_data_folders(data_folder)
num_patients = len(patient_ids)

if num_patients==0:
    raise Exception('No data were provided.')

# Create a folder for the Challenge outputs if it does not already exist.
os.makedirs(output_folder, exist_ok=True)

# Run the team's model(s) on the Challenge data.
print('Running the Challenge models on the Challenge data...')

# Iterate over the patients.
for i in range(num_patients):
    print('    {}/{}...'.format(i+1, num_patients))

    patient_id = patient_ids[i]

    # Allow or disallow the model(s) to fail on parts of the data; this can be helpful for debugging.
    try:
        outcome_binary, outcome_probability, cpc = run_challenge_models(models, data_folder, patient_id, True) ### Teams: Implement this function!!!
    except:
            print('... failed.')
            raise

    # Save Challenge outputs.
    os.makedirs(os.path.join(output_folder, patient_id), exist_ok=True)
    output_file = os.path.join(output_folder, patient_id, patient_id + '.txt')
    save_challenge_outputs(output_file, patient_id, outcome_binary, outcome_probability, cpc)

print('Done.')

Loading the Challenge models...
Finding the Challenge data...
Running the Challenge models on the Challenge data...
    1/122...
    2/122...
    3/122...
    4/122...
    5/122...
    6/122...
    7/122...
    8/122...
    9/122...
    10/122...
    11/122...
    12/122...
    13/122...
    14/122...
    15/122...
    16/122...
    17/122...
    18/122...
    19/122...
    20/122...
    21/122...
    22/122...
    23/122...
    24/122...
    25/122...
    26/122...
    27/122...
    28/122...
    29/122...
    30/122...
    31/122...
    32/122...
    33/122...
    34/122...
    35/122...
    36/122...
    37/122...
    38/122...
    39/122...
    40/122...
    41/122...
    42/122...
    43/122...
    44/122...
    45/122...
    46/122...
    47/122...
    48/122...
    49/122...
    50/122...
    51/122...
    52/122...
    53/122...
    54/122...
    55/122...
    56/122...
    57/122...
    58/122...
    59/122...
    60/122...
    61/122...
    62/122...
    63/122...
    64/122.

# **Evaluation Helper**

In [ ]:
#!/usr/bin/env python

# Do *not* edit this script.
# These are helper functions that you can use with your code.
# Check the example code to see how to import these functions to your code.

import os, numpy as np, scipy as sp, scipy.io

### Challenge data I/O functions

# Find the folders with data files.
def find_data_folders(root_folder):
    data_folders = list()
    for x in sorted(os.listdir(root_folder)):
        data_folder = os.path.join(root_folder, x)
        if os.path.isdir(data_folder):
            data_file = os.path.join(data_folder, x + '.txt')
            if os.path.isfile(data_file):
                data_folders.append(x)
    return sorted(data_folders)

# Load the patient metadata: age, sex, etc.
def load_challenge_data(data_folder, patient_id):
    patient_metadata_file = os.path.join(data_folder, patient_id, patient_id + '.txt')
    patient_metadata = load_text_file(patient_metadata_file)
    return patient_metadata

# Find the record names.
def find_recording_files(data_folder, patient_id):
    record_names = list()
    patient_folder = os.path.join(data_folder, patient_id)
    for file_name in sorted(os.listdir(patient_folder)):
        if not file_name.startswith('.') and file_name.endswith('.hea'):
            root, ext = os.path.splitext(file_name)
            record_name = '_'.join(root.split('_')[:-1])
            record_names.append(record_name)
    return sorted(record_names)

# Load the WFDB data for the Challenge (but not all possible WFDB files).
def load_recording_data(record_name, check_values=True):
    # Allow either the record name or the header filename.
    root, ext = os.path.splitext(record_name)
    if ext=='':
        header_file = record_name + '.hea'
    else:
        header_file = record_name

    # Load the header file.
    if not os.path.isfile(header_file):
        raise FileNotFoundError('{} recording not found.'.format(record_name))

    with open(header_file, 'r') as f:
        header = [l.strip() for l in f.readlines() if l.strip()]

    # Parse the header file.
    record_name = None
    num_signals = None
    sampling_frequency = None
    num_samples = None
    signal_files = list()
    gains = list()
    offsets = list()
    channels = list()
    initial_values = list()
    checksums = list()

    for i, l in enumerate(header):
        arrs = [arr.strip() for arr in l.split(' ')]
        # Parse the record line.
        if i==0:
            record_name = arrs[0]
            num_signals = int(arrs[1])
            sampling_frequency = float(arrs[2])
            num_samples = int(arrs[3])
        # Parse the signal specification lines.
        elif not l.startswith('#') or len(l.strip()) == 0:
            signal_file = arrs[0]
            gain = float(arrs[2].split('/')[0])
            offset = int(arrs[4])
            initial_value = int(arrs[5])
            checksum = int(arrs[6])
            channel = arrs[8]
            signal_files.append(signal_file)
            gains.append(gain)
            offsets.append(offset)
            initial_values.append(initial_value)
            checksums.append(checksum)
            channels.append(channel)

    # Check that the header file only references one signal file. WFDB format allows for multiple signal files, but, for
    # simplicity, we have not done that here.
    num_signal_files = len(set(signal_files))
    if num_signal_files!=1:
        raise NotImplementedError('The header file {}'.format(header_file) \
            + ' references {} signal files; one signal file expected.'.format(num_signal_files))

    # Load the signal file.
    head, tail = os.path.split(header_file)
    signal_file = os.path.join(head, list(signal_files)[0])
    data = np.asarray(sp.io.loadmat(signal_file)['val'])

    # Check that the dimensions of the signal data in the signal file is consistent with the dimensions for the signal data given
    # in the header file.
    num_channels = len(channels)
    if np.shape(data)!=(num_channels, num_samples):
        raise ValueError('The header file {}'.format(header_file) \
            + ' is inconsistent with the dimensions of the signal file.')

    # Check that the initial value and checksums in the signal file are consistent with the initial value and checksums in the
    # header file.
    if check_values:
        for i in range(num_channels):
            if data[i, 0]!=initial_values[i]:
                raise ValueError('The initial value in header file {}'.format(header_file) \
                    + ' is inconsistent with the initial value for channel {} in the signal data'.format(channels[i]))
            if np.sum(data[i, :])!=checksums[i]:
                raise ValueError('The checksum in header file {}'.format(header_file) \
                    + ' is inconsistent with the checksum value for channel {} in the signal data'.format(channels[i]))

    # Rescale the signal data using the gains and offsets.
    rescaled_data = np.zeros(np.shape(data), dtype=np.float32)
    for i in range(num_channels):
        rescaled_data[i, :] = (data[i, :]-offsets[i])/gains[i]

    return rescaled_data, channels, sampling_frequency

# Choose the channels.
def reduce_channels(current_data, current_channels, requested_channels):
    if current_channels == requested_channels:
        reduced_data = current_data
        reduced_channels = current_channels
    else:
        reduced_indices = [current_channels.index(channel) for channel in requested_channels if channel in current_channels]
        reduced_channels = [current_channels[i] for i in reduced_indices]
        reduced_data = current_data[reduced_indices, :]
    return reduced_data, reduced_channels

# Choose the channels.
def expand_channels(current_data, current_channels, requested_channels):
    if current_channels == requested_channels:
        expanded_data = current_data
    else:
        num_current_channels, num_samples = np.shape(current_data)
        num_requested_channels = len(requested_channels)
        expanded_data = np.zeros((num_requested_channels, num_samples))
        for i, channel in enumerate(requested_channels):
            if channel in current_channels:
                j = current_channels.index(channel)
                expanded_data[i, :] = current_data[j, :]
            else:
                expanded_data[i, :] = float('nan')
    return expanded_data

### Helper Challenge data I/O functions

# Load text file as a string.
def load_text_file(filename):
    with open(filename, 'r') as f:
        data = f.read()
    return data

# Get a variable from the patient metadata.
def get_variable(text, variable_name, variable_type):
    variable = None
    for l in text.split('\n'):
        if l.startswith(variable_name):
            variable = ':'.join(l.split(':')[1:]).strip()
            variable = cast_variable(variable, variable_type)
            return variable

# Get the patient ID variable from the patient data.
def get_patient_id(string):
    return get_variable(string, 'Patient', str)

# Get the patient ID variable from the patient data.
def get_hospital(string):
    return get_variable(string, 'Hospital', str)

# Get the age variable (in years) from the patient data.
def get_age(string):
    return get_variable(string, 'Age', int)

# Get the sex variable from the patient data.
def get_sex(string):
    return get_variable(string, 'Sex', str)

# Get the ROSC variable (in minutes) from the patient data.
def get_rosc(string):
    return get_variable(string, 'ROSC', int)

# Get the OHCA variable from the patient data.
def get_ohca(string):
    return get_variable(string, 'OHCA', bool)

# Get the shockable rhythm variable from the patient data.
def get_shockable_rhythm(string):
    return get_variable(string, 'Shockable Rhythm', bool)

# Get the TTM variable (in Celsius) from the patient data.
def get_ttm(string):
    return get_variable(string, 'TTM', int)

# Get the Outcome variable from the patient data.
def get_outcome(string):
    variable = get_variable(string, 'Outcome', str)
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    if variable == 'Good':
        variable = 0
    elif variable == 'Poor':
        variable = 1
    return variable

# Get the Outcome probability variable from the patient data.
def get_outcome_probability(string):
    variable = sanitize_scalar_value(get_variable(string, 'Outcome Probability', str))
    if variable is None or is_nan(variable):
        raise ValueError('No outcome available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the CPC variable from the patient data.
def get_cpc(string):
    variable = sanitize_scalar_value(get_variable(string, 'CPC', str))
    if variable is None or is_nan(variable):
        raise ValueError('No CPC score available. Is your code trying to load labels from the hidden data?')
    return variable

# Get the utility frequency (in Hertz) from the recording data.
def get_utility_frequency(string):
    return get_variable(string, '#Utility frequency', int)

# Get the start time (in hh:mm:ss format) from the recording data.
def get_start_time(string):
    variable = get_variable(string, '#Start time', str)
    times = tuple(int(value) for value in variable.split(':'))
    return times

# Get the end time (in hh:mm:ss format) from the recording data.
def get_end_time(string):
    variable = get_variable(string, '#End time', str)
    times = tuple(int(value) for value in variable.split(':'))
    return times

# Convert seconds to days, hours, minutes, seconds.
def convert_seconds_to_hours_minutes_seconds(seconds):
    hours = int(seconds/3600 - 24*days)
    minutes = int(seconds/60 - 24*60*days - 60*hours)
    seconds = int(seconds - 24*3600*days - 3600*hours - 60*minutes)
    return hours, minutes, seconds

# Convert hours, minutes, and seconds to seconds.
def convert_hours_minutes_seconds_to_seconds(hours, minutes, seconds):
    return 3600*hours + 60*minutes + seconds

### Challenge label and output I/O functions


# Save the Challenge outputs for one file.
def save_challenge_outputs(filename, patient_id, outcome, outcome_probability, cpc):
    # Sanitize values, e.g., in case they are a singleton array.
    outcome = sanitize_boolean_value(outcome)
    outcome_probability = sanitize_scalar_value(outcome_probability)
    cpc = sanitize_scalar_value(cpc)

    # Format Challenge outputs.
    patient_string = 'Patient: {}'.format(patient_id)
    if outcome == 0:
        outcome = 'Good'
    elif outcome == 1:
        outcome = 'Poor'
    outcome_string = 'Outcome: {}'.format(outcome)
    outcome_probability_string = 'Outcome Probability: {:.3f}'.format(outcome_probability)
    cpc_string = 'CPC: {:.3f}'.format(cast_int_if_int_else_float(cpc))
    output_string = patient_string + '\n' + \
        outcome_string + '\n' + outcome_probability_string + '\n' + cpc_string + '\n'

    # Write the Challenge outputs.
    if filename is not None:
        with open(filename, 'w') as f:
            f.write(output_string)

    return output_string

### Other helper functions

# Check if a variable is a number or represents a number.
def is_number(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False

# Check if a variable is an integer or represents an integer.
def is_integer(x):
    if is_number(x):
        return float(x).is_integer()
    else:
        return False

# Check if a variable is a boolean or represents a boolean.
def is_boolean(x):
    if (is_number(x) and float(x)==0) or (remove_extra_characters(x) in ('False', 'false', 'FALSE', 'F', 'f')):
        return True
    elif (is_number(x) and float(x)==1) or (remove_extra_characters(x) in ('True', 'true', 'TRUE', 'T', 't')):
        return True
    else:
        return False

# Check if a variable is a finite number or represents a finite number.
def is_finite_number(x):
    if is_number(x):
        return np.isfinite(float(x))
    else:
        return False

# Check if a variable is a NaN (not a number) or represents a NaN.
def is_nan(x):
    if is_number(x):
        return np.isnan(float(x))
    else:
        return False

# Remove any quotes, brackets (for singleton arrays), and/or invisible characters.
def remove_extra_characters(x):
    return str(x).replace('"', '').replace("'", "").replace('[', '').replace(']', '').replace(' ', '').strip()

# Sanitize boolean values.
def sanitize_boolean_value(x):
    x = remove_extra_characters(x)
    if (is_number(x) and float(x)==0) or (remove_extra_characters(x) in ('False', 'false', 'FALSE', 'F', 'f')):
        return 0
    elif (is_number(x) and float(x)==1) or (remove_extra_characters(x) in ('True', 'true', 'TRUE', 'T', 't')):
        return 1
    else:
        return float('nan')

# Sanitize integer values.
def sanitize_integer_value(x):
    x = remove_extra_characters(x)
    if is_integer(x):
        return int(float(x))
    else:
        return float('nan')

# Sanitize scalar values.
def sanitize_scalar_value(x):
    x = remove_extra_characters(x)
    if is_number(x):
        return float(x)
    else:
        return float('nan')

# Cast a value to a particular type.
def cast_variable(variable, variable_type, preserve_nan=True):
    if preserve_nan and is_nan(variable):
        variable = float('nan')
    else:
        if variable_type == bool:
            variable = sanitize_boolean_value(variable)
        elif variable_type == int:
            variable = sanitize_integer_value(variable)
        elif variable_type == float:
            variable = sanitize_scalar_value(variable)
        else:
            variable = variable_type(variable)
    return variable

# Cast a value to an integer if the value is an integer, a float if the value is a non-integer float, and itself otherwise.
def cast_int_if_int_else_float(x):
    if is_integer(x):
        return int(float(x))
    elif is_number(x):
        return float(x)
    else:
        return x


import os, os.path, sys, numpy as np


# Evaluate the models.
def evaluate_model(label_folder, output_folder):
    # Load the labels.
    patient_ids = find_data_folders(label_folder)
    num_patients = len(patient_ids)

    hospitals = list()
    label_outcomes = list()
    label_cpcs = list()

    for i in range(num_patients):
        patient_data_file = os.path.join(label_folder, patient_ids[i], patient_ids[i] + '.txt')
        patient_data = load_text_file(patient_data_file)

        hospital = get_hospital(patient_data)
        label_outcome = get_outcome(patient_data)
        label_cpc = get_cpc(patient_data)

        hospitals.append(hospital)
        label_outcomes.append(label_outcome)
        label_cpcs.append(label_cpc)

    # Load the model outputs.
    output_outcomes = list()
    output_outcome_probabilities = list()
    output_cpcs = list()

    for i in range(num_patients):
        output_file = os.path.join(output_folder, patient_ids[i], patient_ids[i] + '.txt')
        output_data = load_text_file(output_file)

        output_outcome = get_outcome(output_data)
        output_outcome_probability = get_outcome_probability(output_data)
        output_cpc = get_cpc(output_data)

        output_outcomes.append(output_outcome)
        output_outcome_probabilities.append(output_outcome_probability)
        output_cpcs.append(output_cpc)

    # Evaluate the models.
    challenge_score = compute_challenge_score(label_outcomes, output_outcome_probabilities, hospitals)
    auroc_outcomes, auprc_outcomes = compute_auc(label_outcomes, output_outcome_probabilities)
    accuracy_outcomes, _, _ = compute_accuracy(label_outcomes, output_outcomes)
    f_measure_outcomes, _, _ = compute_f_measure(label_outcomes, output_outcomes)
    mse_cpcs = compute_mse(label_cpcs, output_cpcs)
    mae_cpcs = compute_mae(label_cpcs, output_cpcs)

    # Return the results.
    return challenge_score, auroc_outcomes, auprc_outcomes, accuracy_outcomes, f_measure_outcomes, mse_cpcs, mae_cpcs

# Compute the Challenge score.
def compute_challenge_score(labels, outputs, hospitals):
    # Check the data.
    assert len(labels) == len(outputs)

    # Convert the data to NumPy arrays for easier indexing.
    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)

    # Identify the unique hospitals.
    unique_hospitals = sorted(set(hospitals))
    num_hospitals = len(unique_hospitals)

    # Initialize a confusion matrix for each hospital.
    tps = np.zeros(num_hospitals)
    fps = np.zeros(num_hospitals)
    fns = np.zeros(num_hospitals)
    tns = np.zeros(num_hospitals)

    # Compute the confusion matrix at each output threshold separately for each hospital.
    for i, hospital in enumerate(unique_hospitals):
        idx = [j for j, x in enumerate(hospitals) if x == hospital]
        current_labels = labels[idx]
        current_outputs = outputs[idx]
        num_instances = len(current_labels)

        # Collect the unique output values as the thresholds for the positive and negative classes.
        thresholds = np.unique(current_outputs)
        thresholds = np.append(thresholds, thresholds[-1]+1)
        thresholds = thresholds[::-1]
        num_thresholds = len(thresholds)

        idx = np.argsort(current_outputs)[::-1]

        # Initialize the TPs, FPs, FNs, and TNs with no positive outputs.
        tp = np.zeros(num_thresholds)
        fp = np.zeros(num_thresholds)
        fn = np.zeros(num_thresholds)
        tn = np.zeros(num_thresholds)

        tp[0] = 0
        fp[0] = 0
        fn[0] = np.sum(current_labels == 1)
        tn[0] = np.sum(current_labels == 0)

        # Update the TPs, FPs, FNs, and TNs using the values at the previous threshold.
        k = 0
        for l in range(1, num_thresholds):
            tp[l] = tp[l-1]
            fp[l] = fp[l-1]
            fn[l] = fn[l-1]
            tn[l] = tn[l-1]

            while k < num_instances and current_outputs[idx[k]] >= thresholds[l]:
                if current_labels[idx[k]] == 1:
                    tp[l] += 1
                    fn[l] -= 1
                else:
                    fp[l] += 1
                    tn[l] -= 1
                k += 1

            # Compute the FPRs.
            fpr = np.zeros(num_thresholds)
            for l in range(num_thresholds):
                if tp[l] + fn[l] > 0:
                    fpr[l] = float(fp[l]) / float(tp[l] + fn[l])
                else:
                    fpr[l] = float('nan')

            # Find the threshold such that FPR <= 0.05.
            max_fpr = 0.05
            if np.any(fpr <= max_fpr):
                l = max(l for l, x in enumerate(fpr) if x <= max_fpr)
                tps[i] = tp[l]
                fps[i] = fp[l]
                fns[i] = fn[l]
                tns[i] = tn[l]
            else:
                tps[i] = tp[0]
                fps[i] = fp[0]
                fns[i] = fn[0]
                tns[i] = tn[0]

    # Compute the TPR at FPR <= 0.05 for each hospital.
    tp = np.sum(tps)
    fp = np.sum(fps)
    fn = np.sum(fns)
    tn = np.sum(tns)

    if tp + fn > 0:
        max_tpr = tp / (tp + fn)
    else:
        max_tpr = float('nan')

    return max_tpr

# Compute area under the receiver operating characteristic curve (AUROC) and area under the precision recall curve (AUPRC).
def compute_auc(labels, outputs):
    assert len(labels) == len(outputs)
    num_instances = len(labels)

    # Convert the data to NumPy arrays for easier indexing.
    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)

    # Collect the unique output values as the thresholds for the positive and negative classes.
    thresholds = np.unique(outputs)
    thresholds = np.append(thresholds, thresholds[-1]+1)
    thresholds = thresholds[::-1]
    num_thresholds = len(thresholds)

    idx = np.argsort(outputs)[::-1]

    # Initialize the TPs, FPs, FNs, and TNs with no positive outputs.
    tp = np.zeros(num_thresholds)
    fp = np.zeros(num_thresholds)
    fn = np.zeros(num_thresholds)
    tn = np.zeros(num_thresholds)

    tp[0] = 0
    fp[0] = 0
    fn[0] = np.sum(labels == 1)
    tn[0] = np.sum(labels == 0)

    # Update the TPs, FPs, FNs, and TNs using the values at the previous threshold.
    i = 0
    for j in range(1, num_thresholds):
        tp[j] = tp[j-1]
        fp[j] = fp[j-1]
        fn[j] = fn[j-1]
        tn[j] = tn[j-1]

        while i < num_instances and outputs[idx[i]] >= thresholds[j]:
            if labels[idx[i]] == 1:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Compute the TPRs, TNRs, and PPVs at each threshold.
    tpr = np.zeros(num_thresholds)
    tnr = np.zeros(num_thresholds)
    ppv = np.zeros(num_thresholds)
    for j in range(num_thresholds):
        if tp[j] + fn[j] > 0:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = float('nan')
        if fp[j] + tn[j] > 0:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = float('nan')
        if tp[j] + fp[j] > 0:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = float('nan')

    # Compute AUROC as the area under a piecewise linear function with TPR/sensitivity (x-axis) and TNR/specificity (y-axis) and
    # AUPRC as the area under a piecewise constant with TPR/recall (x-axis) and PPV/precision (y-axis).
    auroc = 0.0
    auprc = 0.0
    for j in range(num_thresholds-1):
        auroc += 0.5 * (tpr[j+1] - tpr[j]) * (tnr[j+1] + tnr[j])
        auprc += (tpr[j+1] - tpr[j]) * ppv[j+1]

    return auroc, auprc

# Construct the one-hot encoding of data for the given classes.
def compute_one_hot_encoding(data, classes):
    num_instances = len(data)
    num_classes = len(classes)

    one_hot_encoding = np.zeros((num_instances, num_classes), dtype=np.bool_)
    unencoded_data = list()
    for i, x in enumerate(data):
        for j, y in enumerate(classes):
            if (x == y) or (is_nan(x) and is_nan(y)):
                one_hot_encoding[i, j] = 1

    return one_hot_encoding

# Compute the binary confusion matrix, where the columns are the expert labels and the rows are the classifier labels for the given
# classes.
def compute_confusion_matrix(labels, outputs, classes):
    assert np.shape(labels) == np.shape(outputs)

    num_instances = len(labels)
    num_classes = len(classes)

    A = np.zeros((num_classes, num_classes))
    for k in range(num_instances):
        for i in range(num_classes):
            for j in range(num_classes):
                if outputs[k, i] == 1 and labels[k, j] == 1:
                    A[i, j] += 1

    return A

# Construct the binary one-vs-rest confusion matrices, where the columns are the expert labels and the rows are the classifier
# for the given classes.
def compute_one_vs_rest_confusion_matrix(labels, outputs, classes):
    assert np.shape(labels) == np.shape(outputs)

    num_instances = len(labels)
    num_classes = len(classes)

    A = np.zeros((num_classes, 2, 2))
    for i in range(num_instances):
        for j in range(num_classes):
            if labels[i, j] == 1 and outputs[i, j] == 1: # TP
                A[j, 0, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 1: # FP
                A[j, 0, 1] += 1
            elif labels[i, j] == 1 and outputs[i, j] == 0: # FN
                A[j, 1, 0] += 1
            elif labels[i, j] == 0 and outputs[i, j] == 0: # TN
                A[j, 1, 1] += 1

    return A

# Compute accuracy.
def compute_accuracy(labels, outputs):
    # Compute the confusion matrix.
    classes = np.unique(np.concatenate((labels, outputs)))
    labels = compute_one_hot_encoding(labels, classes)
    outputs = compute_one_hot_encoding(outputs, classes)
    A = compute_confusion_matrix(labels, outputs, classes)

    # Compute accuracy.
    if np.sum(A) > 0:
        accuracy = np.trace(A) / np.sum(A)
    else:
        accuracy = float('nan')

    # Compute per-class accuracy.
    num_classes = len(classes)
    per_class_accuracy = np.zeros(num_classes)
    for i in range(num_classes):
        if np.sum(labels[:, i]) > 0:
            per_class_accuracy[i] = A[i, i] / np.sum(A[:, i])
        else:
            per_class_accuracy[i] = float('nan')

    return accuracy, per_class_accuracy, classes

# Compute macro F-measure.
def compute_f_measure(labels, outputs):
    # Compute confusion matrix.
    classes = np.unique(np.concatenate((labels, outputs)))
    labels = compute_one_hot_encoding(labels, classes)
    outputs = compute_one_hot_encoding(outputs, classes)
    A = compute_one_vs_rest_confusion_matrix(labels, outputs, classes)

    num_classes = len(classes)
    per_class_f_measure = np.zeros(num_classes)
    for k in range(num_classes):
        tp, fp, fn, tn = A[k, 0, 0], A[k, 0, 1], A[k, 1, 0], A[k, 1, 1]
        if 2 * tp + fp + fn > 0:
            per_class_f_measure[k] = float(2 * tp) / float(2 * tp + fp + fn)
        else:
            per_class_f_measure[k] = float('nan')

    if np.any(np.isfinite(per_class_f_measure)):
        macro_f_measure = np.nanmean(per_class_f_measure)
    else:
        macro_f_measure = float('nan')

    return macro_f_measure, per_class_f_measure, classes

# Compute mean-squared error.
def compute_mse(labels, outputs):
    assert len(labels) == len(outputs)

    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)
    mse = np.mean((labels - outputs)**2)

    return mse

# Compute mean-absolute error.
def compute_mae(labels, outputs):
    assert len(labels) == len(outputs)

    labels = np.asarray(labels, dtype=np.float64)
    outputs = np.asarray(outputs, dtype=np.float64)
    mae = np.mean(np.abs(labels - outputs))

    return mae


# **Evaluation**

In [ ]:
scores = evaluate_model(test_path, output_folder)

# Unpack the scores.
challenge_score, auroc_outcomes, auprc_outcomes, accuracy_outcomes, f_measure_outcomes, mse_cpcs, mae_cpcs = scores

# Construct a string with scores.
output_string = \
    'Challenge Score: {:.3f}\n'.format(challenge_score) + \
    'Outcome AUROC: {:.3f}\n'.format(auroc_outcomes) + \
    'Outcome AUPRC: {:.3f}\n'.format(auprc_outcomes) + \
    'Outcome Accuracy: {:.3f}\n'.format(accuracy_outcomes) + \
    'Outcome F-measure: {:.3f}\n'.format(f_measure_outcomes) + \
    'CPC MSE: {:.3f}\n'.format(mse_cpcs) + \
    'CPC MAE: {:.3f}\n'.format(mae_cpcs)

In [ ]:
print(output_string)

Challenge Score: 0.338
Outcome AUROC: 0.743
Outcome AUPRC: 0.803
Outcome Accuracy: 0.713
Outcome F-measure: 0.677
CPC MSE: 2.886
CPC MAE: 1.421

